In [1]:
from website.app import init_app

init_app()

[website.addons.osfstorage.settings]  WARNING: No local.py settings file found
[website.addons.wiki.settings]  WARNING: No local.py settings file found
[root]  DEBUG: Setting storage backends
[root]  INFO: Sentry disabled; Flask's debug mode enabled


<Flask 'framework.flask'>

In [ ]:
# LOAD BLACKLIST GUIDS
from django.db import transaction
from framework.guid.model import BlacklistGuid
from osf_models.models import BlackListGuid
import gevent

odm_blacklist = BlacklistGuid.find()
total = len(odm_blacklist)
count = 0
page_size = 20000


def migrate_blacklist_item(guid):
    pg_guid = BlackListGuid.objects.create(guid=guid._id)

while count < total:
    with transaction.atomic():
        page = odm_blacklist[count:count+page_size]
        threads = []
        for guid in page:
            threads.append(gevent.spawn(migrate_blacklist_item, guid))
            count += 1
        gevent.joinall(threads)
        print 'Committing {} through {}'.format(count-page_size, count)
    
print count
print total

In [ ]:
# LOAD GUIDS
from django.db import transaction
from framework.guid.model import Guid as MODMGuid
from osf_models.models import Guid
import gevent


guids = MODMGuid.find()
total = len(guids)
count = 0
page_size = 20000

def migrate_guid(guid):
    return Guid.objects.create(guid=guid)

while count < total:
    with transaction.atomic():
        threads = []
        for guid in guids[count:count+page_size].get_keys():
            threads.append(gevent.spawn(migrate_guid, guid))
            count += 1
            if count % page_size == 0:
                print count
        gevent.joinall(threads)
        print 'Committing {} through {}'.format(count-page_size, count)
        
print total
print count

In [ ]:
# VERIFY GUIDS
from framework.guid.model import Guid as MODMGuid
from osf_models.models import Guid
import gevent
import copy

modm_guids = MODMGuid.find()
print 'MODM Guids: {}'.format(len(modm_guids))

def get_guid(key):
    try:
        guid = Guid.objects.get(guid=key)
    except Guid.DoesNotExist:
        print 'Guid {} DoesNotExist'.format(key)

threads = []
for modm_guid in modm_guids.get_keys():
    threads.append(gevent.spawn(get_guid, copy.copy(modm_guid)))

gevent.joinall(threads)

print 'Postgres Guids: {}'.format(Guid.objects.count())

In [15]:
# MIGRATE NODES

from website.models import Node as MODMNode
from osf_models.models import Node
from modularodm import Q as MODMQ
from django.core.serializers.json import DjangoJSONEncoder
import json
from django.db import IntegrityError

modm_nodes = MODMNode.find()[:10]

total = len(modm_nodes)
count = 0
print total

fk_node_fields = [
    'forked_from',
    'registered_from',
    'root',
    'parent_node',
    'template_node'
]
m2m_node_fields = [
    'nodes',
]
fk_user_fields = [
    'registered_user',
    'creator',
    'merged_by'
]
m2m_user_fields = [
    'permissions',
    'recently_added',
    'users_watching_node'
]
m2m_tag_fields = [
    'tags',
    'system_tags'
]

def process_node_fk_fields(modm_object):
    fk_nodes = {}
    for fk_node_field in fk_node_fields:
        value = getattr(modm_object, fk_node_field, None)
        if value is not None:
            if value is not modm_object:
                node = get_or_create_node(value)
                if node is not None:
                    fk_nodes[fk_node_field] = node
    return fk_nodes

def process_node_m2m_fields(modm_object):
    m2m_nodes = {}
    for m2m_node_field in m2m_node_fields:
        value = getattr(modm_object, m2m_node_field, None)
        if value is not None:
            if isinstance(value, list):
                for nv in value:
                    if m2m_node_field in m2m_nodes:
                        node = get_or_create_node(nv)
                        if node is not None:
                            m2m_nodes[m2m_node_field].append(node)
                        else:
                            m2m_nodes[m2m_node_field] = [node, ]
            else:
                if m2m_node_field in m2m_nodes:
                    node = get_or_create_node(value)
                    if node is not None:
                        m2m_nodes[m2m_node_field].append(node)
                    else:
                        m2m_nodes[m2m_node_field] = [node, ]
    return fk_nodes

def get_or_create_user(modm_user, field_name, modm_node=None):
    try:
        user = User.objects.get()
    except User.DoesNotExist:
        user = User()
        user_fk_nodes = process_node_fk_fields(modm_user)

def get_or_create_tag(modm_tag, field_name, modm_node=None):
    try:
        tag = Tag.objects.get(_id=modm_tag._id)
    except Tag.DoesNotExist:
        tag = Tag(**modm_tag.to_storage())
    return tag

def get_or_create_node(modm_node):
    try:
        # try and get the node
        node = Node.objects.get(_guid__guid=modm_node._id)
    except Node.DoesNotExist:  
        # if it doesn't exist, check to see if the guid does
        try:
            guid = Guid.objects.get(guid=modm_node._id)
        except Guid.DoesNotExist:
            # fail if the guid doesn't exist
            print 'GUID {} DoesNotExist'.format(modm_guid)
        else:
            fk_nodes = process_node_fk_fields(modm_node)
            
            m2m_nodes = process_node_m2m_fields(modm_node)
            
            fk_users = {}
            for fk_user_field in fk_user_fields:
                modm_user = getattr(modm_node, fk_user_field, None)
                if modm_user is not None:
                    user = get_or_create_user(modm_user, fk_user_field, modm_node)
                    if not user is None:
                        fk_users[fk_user_field] = user
                    
            m2m_users = {}
            for m2m_user_field in m2m_user_fields:
                value = getattr(modm_node, m2m_user_field, None)
#                 print 'm2m_user_field: {} Value {}'.format(m2m_user_field, value)
                if isinstance(value, list):
                    for uv in value:
                        if m2m_user_field in m2m_users:
                            user = get_or_create_user(uv)
                            if user is not None:
                                m2m_users[m2m_user_field].append(user)
                            else:
                                 m2m_users[m2m_user_field] = [user, ]
                                    
            m2m_tags = {}
            for m2m_tag_field in m2m_tag_fields:
                value = getattr(modm_node, m2m_tag_field, None)
                if isinstance(value, list):
                    for tv in value:
                        if m2m_tag_field in m2m_tags:
                            tag = get_or_create_tag(tv)
                            if tag is not None:
                                m2m_tags[m2m_tag_field].append(user)
                            else:
                                m2m_tags[m2m_tag_field] - [tag, ]
#             print 'fk_nodes ', fk_nodes
#             print 'm2m_nodes ', m2m_nodes
#             print 'fk_users ', fk_users
#             print 'm2m_users ', m2m_users
            node_fields = {}
            node_fields.update(modm_node.to_storage())
            node_fields.update(fk_nodes)
            node_fields.update(m2m_nodes)
            node_fields.update(fk_users)
            node_fields.update(m2m_users)
            return Node(**node_fields)
                
    else:
        return node

for modm_node in modm_nodes:
    noooood = get_or_create_node(modm_node)
    print noooood
    count +=1
    if count % 1000 == 0:
        print count
            
print 'MODM: {}'.format(total)
print 'PG: {}'.format(count)

10
USER:  Ben B. Blohowiak  Field:  creator


ValueError: Cannot assign "<User(u'benb@virginia.edu') with id u'vnhtt'>": "Node.creator" must be a "User" instance.

In [ ]:
# MIGRATE REGISTRATIONS

from website.models import Node as MODMNode
from osf_models.models import Registration
from modularodm import Q as MODMQ
from django.core.serializers.json import DjangoJSONEncoder
import json
from django.db import IntegrityError

modm_nodes = MODMNode.find(
    MODMQ('is_registration', 'eq', True) &
    MODMQ('is_folder', 'ne', True)
)

total = len(modm_nodes)
count = 0
print total

for modm_node in modm_nodes:
    try:
        guid = Guid.objects.get(guid=modm_node._id)
    except Guid.DoesNotExist:
        print 'Guid {} DoesNotExist'.format(modm_guid)
    else:
        try:
            node = Registration.objects.create(guid=guid, data=modm_node.to_storage())
        except IntegrityError as ex:
            print 'Registration existed: {}'.format(ex)
        count +=1
        if count % 100 == 0:
            print count
            
print 'MODM: {}'.format(total)
print 'PG: {}'.format(count)

In [ ]:
# MIGRATE COLLECTIONS

from website.models import Node as MODMNode
from osf_models.models import Collection
from modularodm import Q as MODMQ
from django.core.serializers.json import DjangoJSONEncoder
import json
from django.db import IntegrityError

modm_nodes = MODMNode.find(
    MODMQ('is_folder', 'eq', True)
)

total = len(modm_nodes)
count = 0
print total

for modm_node in modm_nodes:
    try:
        guid = Guid.objects.get(guid=modm_node._id)
    except Guid.DoesNotExist:
        print 'Guid {} DoesNotExist'.format(modm_guid)
    else:
        try:
            node = Collection.objects.create(guid=guid, data=modm_node.to_storage())
        except IntegrityError as ex:
            print 'Collection existed: {}'.format(ex)
        count +=1
        if count % 100 == 0:
            print count
            
print 'MODM: {}'.format(total)
print 'PG: {}'.format(count)

In [ ]:
# BENCHMARK /users/me/nodes/

import timeit

modm = """
from website.models import Node
from modularodm import Q
nodes = Node.find(Q('contributors','eq','cdi38') & Q('is_deleted','eq', False) & Q('is_folder','eq', False))[:60]
for node in nodes:
    id = node._id
"""
django = """
from osf_models.models import Node
nodes = Node.objects.filter(data__contributors__contains='"cdi38"', data__contains='{"is_deleted": false, "is_folder": false}')[:60]
for node in nodes:
    id = node.guid_id
    id = node.guid.id
"""
print "django, mongo"
for i in range(0,20):
    d = timeit.timeit(stmt=django, number=50)
    m = timeit.timeit(stmt=modm, number=50)
    print '{} {}'.format(d, m)

In [ ]:
from osf_models.models import Node
nodes = Node.objects.filter(data__contributors__contains='"cdi38"')
print nodes.query
print nodes.count()

In [ ]:
mia_guids = Guid.objects.filter(referent_node=None, referent_registration=None, referent_collection=None).count()
accounted_for_guids = Guid.objects.exclude(referent_node=None, referent_registration=None, referent_collection=None).count()
total_guids = Guid.objects.all().count()
total_guids == accounted_for_guids + mia_guids

